In [1]:
#Install if errors occur.  First run this cell, then reset runtime and start run from cell below.
!pip install --force-reinstall 'sqlalchemy<2.0.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.19
    Uninstalling SQLAlchemy-2.0.19:
      Successfully uninstalled SQLAlchemy-2.0.19


In [2]:
import os


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

# Create an engine object
engine = create_engine('sqlite:///drive/MyDrive/schools_db', echo=False)
connection = engine

# Fetch all records from schools table
df = pd.read_sql_table('schools', connection)
print(df.head())

# Close the Database connection
engine.dispose()

       ID                                 Name    State     Region  \
0  100654             Alabama A & M University  Alabama  Southeast   
1  100663  University of Alabama at Birmingham  Alabama  Southeast   
2  100706  University of Alabama in Huntsville  Alabama  Southeast   
3  100724             Alabama State University  Alabama  Southeast   
4  100751            The University of Alabama  Alabama  Southeast   

   Applications  Admissions  Admission_Rate  Enrollment  SAT_Verbal_75  \
0          6560        4697        0.716006        1459            520   
1         11906       10541        0.885352        2361            680   
2          5781        4259        0.736724        1191            700   
3          5974        5854        0.979913         797            534   
4         42421       33472        0.789043        7540            680   

   SAT_Math_75  ACT_English_75  ACT_Math_75  
0          510              20           18  
1          708              33           2

In [5]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd



In [7]:
# Drop the non-beneficial ID columns, 'SAT_Verbal_75' and 'SAT_Math_75'.
df = df.drop(columns = ['SAT_Verbal_75', 'SAT_Math_75', 'ID', 'Name', 'Admissions', 'Applications'])

In [8]:
# Determine the number of unique values in each column.
for x in df.columns:
    print(x, len(df[x].unique()))

State 51
Region 8
Admission_Rate 1353
Enrollment 920
ACT_English_75 23
ACT_Math_75 24


In [9]:
# Look at Regions value counts for binning
regionbinning = df['Region'].value_counts()

In [10]:
# You may find it helpful to look at Regions value counts >1
regionclassification = regionbinning[regionbinning>1]
regionclassification

Southeast          344
Mid East           271
Great Lakes        215
Plains             143
Far West           131
New England        123
Southwest           94
Rocky Mountains     34
Name: Region, dtype: int64

In [11]:
# Choose a cutoff value and create a list of regions to be replaced
# use the variable name `regions_to_replace`
regions_to_replace = list (regionbinning[regionbinning<100].index)
regions_to_replace

# Replace in dataframe
for reg in regions_to_replace:
    df['Region'] = df['Region'].replace(reg,"Other")

# Check to make sure binning was successful
df['Region'].value_counts()

Southeast      344
Mid East       271
Great Lakes    215
Plains         143
Far West       131
Other          128
New England    123
Name: Region, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df,dtype=float)
df.head()

,Admission_Rate,Enrollment,ACT_English_75,ACT_Math_75,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,...,State_West Virginia,State_Wisconsin,State_Wyoming,Region_Far West,Region_Great Lakes,Region_Mid East,Region_New England,Region_Other,Region_Plains,Region_Southeast
0,0.716006,1459,20,18,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.885352,2361,33,28,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.736724,1191,32,29,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.979913,797,22,19,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.789043,7540,33,28,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = df['Admission_Rate'].values

#drop column IS_SUCCESSFUL
X = df.drop('Admission_Rate', axis=1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    number_input_features = len( X_train_scaled[0])
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=20,
        step=2), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=20,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [16]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [17]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 12s]
val_accuracy: 0.0029498524963855743

Best val_accuracy So Far: 0.0029498524963855743
Total elapsed time: 00h 11m 36s


In [25]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 4,
 'units_0': 15,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 3,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1,
 'units_3': 1}

In [18]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - loss: 0.6612 - accuracy: 0.0029 - 305ms/epoch - 28ms/step
Loss: 0.6612255573272705, Accuracy: 0.0029498524963855743


In [34]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 62        
                                                                 
 dense_1 (Dense)             (None, 15)                30        
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
 dense_3 (Dense)             (None, 1)                 2         
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
 dense_5 (Dense)             (None, 1)                 2         
                                                                 
Total params: 114
Trainable params: 114
Non-trainable pa